<img src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/solutions-microsoft-logo-small.png?raw=true" alt="Microsoft">
<br>

# **SQL Server 2019 big data cluster Tutorial**
## **05 - Creating and Querying a Data Mart**

In this tutorial you will learn how to create and query a Data Mart using Virtualized Data in a SQL Server big data cluster. 

Wide World Importers is interested in ingesting the data from web logs from an HDFS source where they have been streamed. They want to be able to analyze the traffic to see if there is a pattern in time, products or locations. 

The web logs, however, are refreshed periodically. WWI would like to keep the logs in local storage to do deeper analysis. 

In this Jupyter Notebook you'll create a location to store the log files as a SQL Server Table in the SQL Data Pool, and then fill it by creating an External Table that reads HDFS.

In [ ]:
USE WideWorldImporters;
GO

IF NOT EXISTS(SELECT * FROM sys.external_data_sources WHERE name = 'SqlDataPool')
BEGIN
    CREATE EXTERNAL DATA SOURCE SqlDataPool
    WITH (LOCATION = 'sqldatapool://controller-svc/default');
END


In [ ]:
CREATE EXTERNAL TABLE [web_clickstream_clicks_data_pool]
    ("wcs_click_date_sk" BIGINT 
    , "wcs_click_time_sk" BIGINT 
    , "wcs_sales_sk" BIGINT 
    , "wcs_item_sk" BIGINT
    , "wcs_web_page_sk" BIGINT 
    , "wcs_user_sk" BIGINT)
    WITH
    (
    DATA_SOURCE = SqlDataPool,
    DISTRIBUTION = ROUND_ROBIN
    );
GO

In [ ]:
/* Create an External Table that can read from the Storage Pool File Location */
IF NOT EXISTS(SELECT * FROM sys.external_tables WHERE name = 'web_clickstreams_hdfs')
BEGIN
    CREATE EXTERNAL TABLE [web_clickstreams_hdfs]
    ("wcs_click_date_sk" BIGINT 
    , "wcs_click_time_sk" BIGINT 
    , "wcs_sales_sk" BIGINT 
    , "wcs_item_sk" BIGINT
    , "wcs_web_page_sk" BIGINT 
    , "wcs_user_sk" BIGINT)
    WITH
    (
        DATA_SOURCE = SqlStoragePool,
        LOCATION = '/web_logs',
        FILE_FORMAT = csv_file
    );
END

In [ ]:
/* Insert data into data pool from storage pool*/
BEGIN
   INSERT INTO web_clickstream_clicks_data_pool
   SELECT wcs_click_date_sk
    , wcs_click_time_sk 
    , wcs_sales_sk 
    , wcs_item_sk 
    , wcs_web_page_sk  
    , wcs_user_sk 
     FROM web_clickstreams_hdfs
END

You can visualize the sales data inside azure data studio.  
- Execute the below cell and click on the second dataframe. 

![plot](images/plot.png)

---
- Change the configuration of the plot, and you'll see the sales trend plot.

![plot-config](images/plot-config.png)

In [ ]:
/* SELECT clickstream data so that you can visualize it*/
SELECT count(*) FROM [dbo].[web_clickstream_clicks_data_pool];
SELECT TOP 1000 wcs_click_date_sk, wcs_sales_sk FROM [dbo].[web_clickstream_clicks_data_pool]
WHERE wcs_sales_sk IS NOT NULL

Clean up only - run this cell only if you are repeating the demo!

In [ ]:
USE WideWorldImporters;

IF EXISTS(SELECT * FROM sys.external_tables WHERE name = 'web_clickstream_clicks_data_pool')
BEGIN
    DROP EXTERNAL TABLE web_clickstream_clicks_data_pool
END

IF EXISTS(SELECT * FROM sys.external_tables WHERE name = 'web_clickstreams_hdfs')
BEGIN
    DROP EXTERNAL TABLE web_clickstreams_hdfs
END

## **Next Steps: Continue on to Working with Spark and ETL**

Now you're ready to open the next Jupyter Notebook - [bdc-06-spark-etl.ipynb](bdc-06-spark-etl.ipynb) - to learn how to create and work with Spark and Extracting, Transforming and Loading data.